In [2]:
import torch
from torch.utils.data import DataLoader
import os
import numpy as np
from PIL import Image
import open3d as o3d
from dataset import *

/Users/rishabhrao/Documents/VSCode/multimodal-3d-gen/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define paths
dataset_path = "Data/ShapeNetSem/Datasets/subset_template_200.csv"  # CSV file containing mesh IDs
pc_dir = "Data/ProcessedData/PointClouds1024"  # Directory containing .npy files

# Initialize dataset
dataset = PCModalityDataset(dataset_path, pc_dir)

# Initialize DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

print(f"Dataset size: {len(dataset)} samples")

Dataset size: 200 samples


In [4]:
import open3d as o3d
import numpy as np
import torch
from PIL import Image

# Convert 3D Point Clouds to 2D Depth Maps
def point_cloud_to_depth_map(pc_data):
    """Converts a 3D point cloud to a 2D depth image."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc_data.numpy())

    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.add_geometry(pcd)

    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    vis.poll_events()
    vis.update_renderer()

    # Capture depth image
    depth = vis.capture_depth_float_buffer(True)
    vis.destroy_window()

    # Convert depth to PIL Image
    depth_np = np.asarray(depth)
    depth_image = Image.fromarray((depth_np * 255).astype(np.uint8))

    return depth_image

In [5]:
"""
import torch
import open_clip

# Define Model
model_name = "ViT-B-32"  # Standard for PointCLIP
pretrained = "openai"  # Use OpenAI's pretrained CLIP

# Load Model
clip_model, _, _ = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)

# Save Model Locally
save_path = "PretrainedModels/clip_vitb32_pretrain.pth"
torch.save(clip_model.state_dict(), save_path)

print(f"✅ CLIP Model Saved to: {save_path}")"""

'\nimport torch\nimport open_clip\n\n# Define Model\nmodel_name = "ViT-B-32"  # Standard for PointCLIP\npretrained = "openai"  # Use OpenAI\'s pretrained CLIP\n\n# Load Model\nclip_model, _, _ = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)\n\n# Save Model Locally\nsave_path = "PretrainedModels/clip_vitb32_pretrain.pth"\ntorch.save(clip_model.state_dict(), save_path)\n\nprint(f"✅ CLIP Model Saved to: {save_path}")'

In [6]:
import torch
import numpy as np
import open_clip
from PIL import Image
import open3d as o3d
from torch.utils.data import DataLoader

# Define Model Path
save_path = "PretrainedModels/clip_vitb32_pretrain.pth"
model_name = "ViT-B-32"

# Load Model Without Downloading
clip_model = open_clip.create_model(model_name, pretrained=False)
checkpoint = torch.load(save_path, map_location="cpu")
clip_model.load_state_dict(checkpoint)
clip_model.eval()

# Move Model to GPU if Available
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model.to(device)

print("✅ CLIP Model Loaded Successfully!")

✅ CLIP Model Loaded Successfully!


In [7]:
import torch
import open_clip
import numpy as np
from PIL import Image
import open3d as o3d
from torch.utils.data import DataLoader
from open_clip import image_transform

# Define Model Path
save_path = "PretrainedModels/clip_vitb32_pretrain.pth"
model_name = "ViT-B-32"

# Load Model Without Downloading
clip_model = open_clip.create_model(model_name, pretrained=False)
checkpoint = torch.load(save_path, map_location="cpu")
clip_model.load_state_dict(checkpoint)
clip_model.eval()

# Move Model to GPU if Available
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model.to(device)

print("✅ CLIP Model Loaded Successfully!")

# Convert 3D Point Clouds to 2D Depth Maps
def point_cloud_to_depth_map(pc_data):
    """Converts a 3D point cloud to a 2D depth image."""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc_data.numpy())

    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False)
    vis.add_geometry(pcd)

    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    vis.poll_events()
    vis.update_renderer()

    # Capture depth image
    depth = vis.capture_depth_float_buffer(True)
    vis.destroy_window()

    # Convert depth to PIL Image
    depth_np = np.asarray(depth)
    depth_image = Image.fromarray((depth_np * 255).astype(np.uint8))

    return depth_image

# Load Dataset
dataset_path = "Data/ShapeNetSem/Datasets/subset_template_200.csv"
pc_dir = "Data/ProcessedData/PointClouds1024"  # Update to correct directory

pc_dataset = PCModalityDataset(dataset_path, pc_dir)
dataloader = DataLoader(pc_dataset, batch_size=1, shuffle=False, num_workers=0)

# Set up CLIP preprocessing
preprocess = image_transform(
    clip_model.visual.image_size,  # Correct image size for CLIP
    is_train=False  # Ensures we use inference preprocessing
)

# Extract 3D Point Cloud Embeddings
for idx, pc_data in dataloader:
    pc_data = pc_data.to(device)  # Move to device

    # Convert point cloud to depth map
    depth_map = point_cloud_to_depth_map(pc_data[0])

    # Preprocess Depth Map for CLIP
    depth_map = preprocess(depth_map).unsqueeze(0).to(device)

    # Encode Image using CLIP
    with torch.no_grad():
        embedding = clip_model.encode_image(depth_map)

    print(f"Point Cloud {idx}: Embedding Shape: {embedding.shape}")  # Expected: (1, 512)
    break  # Process one batch for testing


✅ CLIP Model Loaded Successfully!
Point Cloud tensor([0]): Embedding Shape: torch.Size([1, 512])


In [8]:
from torch import Tensor
# Load Dataset
# Load Dataset
dataset_path = "Data/ShapeNetSem/Datasets/subset_template_200.csv"
pc_dir = "Data/ProcessedData/PointClouds1024"  # Update to correct directory

pc_dataset = PCModalityDataset(dataset_path, pc_dir)
dataloader = DataLoader(pc_dataset, batch_size=1, shuffle=False, num_workers=0)

# Set up CLIP preprocessing
preprocess = image_transform(
    clip_model.visual.image_size,  # Correct image size for CLIP
    is_train=False  # Ensures we use inference preprocessing
)

all_embeddings = []
all_idx = []
all_cats = []
data = pd.read_csv(dataset_path)


for idx, pc_data in dataloader:
    pc_data = pc_data.to(device)  # Move to device

    # Convert point cloud to depth map
    depth_map = point_cloud_to_depth_map(pc_data[0])

    # Preprocess Depth Map for CLIP
    depth_map = preprocess(depth_map).unsqueeze(0).to(device)

    # Encode Image using CLIP
    with torch.no_grad():
        embedding = clip_model.encode_image(depth_map)

    all_embeddings.append(embedding)
    all_idx.append(idx)

    all_cats.append(data.loc[int(idx.item()), 'category'])


In [9]:
len(all_embeddings), len(all_idx), len(all_cats)

(200, 200, 200)

In [11]:
embeddings_t = torch.concat(all_embeddings, dim=0)
data_dict = {
    "embedding": embeddings_t,
    "index": all_idx,
    "category": all_cats
}

save_path = "Embeddings/PointCLIP/subset_template_200.pt"
torch.save(data_dict, save_path)

In [12]:
import sklearn

In [13]:
embeddings_t.numpy().shape

(200, 512)

In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
import numpy as np

embeddings = embeddings_t.numpy()

In [15]:
formatted_categories = []
for subcategory in all_cats:
    subcategories = subcategory.split(',')
    new_subcategories = [s for s in subcategories if '_' not in s]
    formatted_categories.append(new_subcategories[0])

In [16]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
categories = np.array(formatted_categories).reshape(-1, 1)
# Fit and transform the data
one_hot_encoded = encoder.fit_transform(categories)
print(f"After OHE categories shape: {one_hot_encoded.shape}")
labels = one_hot_encoded.argmax(axis=1)
print(f"After Argmax labels shape: {labels.shape}")
cat_mapping = {i: cat for i, cat in enumerate(encoder.categories_[0])}
print("Mapping (integer label -> string category):", cat_mapping)


After OHE categories shape: (200, 19)
After Argmax labels shape: (200,)
Mapping (integer label -> string category): {0: np.str_('Bed'), 1: np.str_('Books'), 2: np.str_('Camera'), 3: np.str_('CeilingFan'), 4: np.str_('ChestOfDrawers'), 5: np.str_('Couch'), 6: np.str_('DrinkingUtensil'), 7: np.str_('Fan'), 8: np.str_('Faucet'), 9: np.str_('MediaStorage'), 10: np.str_('PersonStanding'), 11: np.str_('PillBottle'), 12: np.str_('Plant'), 13: np.str_('PottedPlant'), 14: np.str_('Showerhead'), 15: np.str_('Table'), 16: np.str_('Truck'), 17: np.str_('USBStick'), 18: np.str_('Vase')}


In [23]:
print(f"Before PCA embedding shape: {embeddings.shape}")
pca = PCA(n_components=25, random_state=42)
pca_result = pca.fit_transform(embeddings)
print(f"After PCA embedding shape: {pca_result.shape}")

Before PCA embedding shape: (200, 512)
After PCA embedding shape: (200, 25)


In [24]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=42)
print(f"Before TSNE embedding shape: {pca_result.shape}")
tsne_result = tsne.fit_transform(pca_result)
print(f"After TSNE embedding shape: {tsne_result.shape}")

Before TSNE embedding shape: (200, 25)
After TSNE embedding shape: (200, 2)


In [25]:
import plotly.express as px
plot_data = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'numeric_label': labels,
    'category': [cat_mapping[label] for label in labels]
})

# Create an interactive scatter plot using Plotly Express
fig = px.scatter(
    plot_data, x='x', y='y', color='category',
    hover_data={'numeric_label': True, 'category': True},
    title="Interactive t-SNE Visualization with Category Info"
)
fig.show()